In [3]:
import gcsfs
import xarray as xr
import numpy as np
from scale_aware_air_sea.cm26_utils import load_and_combine_cm26
from scale_aware_air_sea.utils import smooth_inputs_dataset, to_zarr_split

In [10]:
fs = gcsfs.GCSFileSystem()

version = 'v0.6.1' # switch to 'v0.6' to get the results from the older (narrower transition width filter)
suffix = '_test'

# set up save targets
bucket = 'gs://leap-persistent/jbusecke'

smooth_path = f"{bucket}/scale-aware-air-sea/smoothed/CM26_smoothed_{version}{suffix}.zarr"
smooth_mapper = fs.get_mapper(smooth_path)
flux_path = f"{bucket}/scale-aware-air-sea/results/CM26_fluxes_{version}{suffix}.zarr"
flux_mapper = fs.get_mapper(flux_path)
decomposition_path = f'leap-scratch/jbusecke/scale-aware-air-sea/decomposition/CM26_decomposed_{version}{suffix}.zarr'
decomposition_mapper = fs.get_mapper(decomposition_path)

## The raw dataset (all interpolated on the ocean tracer grid)

In [5]:
ds = load_and_combine_cm26(fs)
ds

Load Data
Align in time
Interpolating ocean velocities
Regrid Atmospheric Data
Modify units
Mask nans
Calculate relative wind


<xarray.Dataset>
Dimensions:       (time: 7305, yt_ocean: 2700, xt_ocean: 3600)
Coordinates:
  * time          (time) object 0181-01-01 12:00:00 ... 0200-12-31 12:00:00
  * xt_ocean      (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean      (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
Data variables:
    slp           (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_ref         (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_ref         (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    t_ref         (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    q_ref         (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    wind          (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    surface_temp  (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_ocean       (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_ocean       (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_relative    (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_relative    (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

## The smoothed fields

In [6]:
ds_smooth = xr.open_dataset(smooth_mapper, engine='zarr')
ds_smooth

<xarray.Dataset>
Dimensions:                         (time: 3, yt_ocean: 2700, xt_ocean: 3600)
Coordinates:
  * time                            (time) object 0181-01-01 12:00:00 ... 018...
  * xt_ocean                        (xt_ocean) float64 -279.9 -279.8 ... 79.95
  * yt_ocean                        (yt_ocean) float64 -81.11 -81.07 ... 89.98
Data variables:
    q_ref_smoothed                  (time, yt_ocean, xt_ocean) float32 ...
    slp_smoothed                    (time, yt_ocean, xt_ocean) float32 ...
    surface_temp_smoothed           (time, yt_ocean, xt_ocean) float32 ...
    t_ref_smoothed                  (time, yt_ocean, xt_ocean) float32 ...
    u_relative_smoothed             (time, yt_ocean, xt_ocean) float32 ...
    u_relative_smoothed_atmos_only  (time, yt_ocean, xt_ocean) float32 ...
    u_relative_smoothed_ocean_only  (time, yt_ocean, xt_ocean) float32 ...
    v_relative_smoothed             (time, yt_ocean, xt_ocean) float32 ...
    v_relative_smoothed_atmos_only  (time, yt_ocean, xt_ocean) float32 ...
    v_relative_smoothed_ocean_only  (time, yt_ocean, xt_ocean) float32 ...
Attributes:
    regrid_method:  bilinear

## The recomputed fluxes, with differently smoothed input

In [7]:
ds_flux = xr.open_dataset(flux_mapper, engine='zarr')
ds_flux

<xarray.Dataset>
Dimensions:    (algo: 3, smoothing: 7, time: 3, yt_ocean: 2700, xt_ocean: 3600)
Coordinates:
  * algo       (algo) <U8 'ncar' 'ecmwf' 'coare3p6'
  * smoothing  (smoothing) <U23 'smooth_none' 'smooth_tracer' ... 'smooth_all'
  * time       (time) object 0181-01-01 12:00:00 ... 0181-01-03 12:00:00
  * xt_ocean   (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean   (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
Data variables:
    evap       (algo, smoothing, time, yt_ocean, xt_ocean) float32 ...
    qh         (algo, smoothing, time, yt_ocean, xt_ocean) float32 ...
    ql         (algo, smoothing, time, yt_ocean, xt_ocean) float32 ...
    taux       (algo, smoothing, time, yt_ocean, xt_ocean) float32 ...
    tauy       (algo, smoothing, time, yt_ocean, xt_ocean) float32 ...

## The decomposed terms

In [13]:
ds_decomposition = xr.open_dataset(decomposition_mapper, engine='zarr')
ds_decomposition

<xarray.Dataset>
Dimensions:   (term: 20, time: 3, yt_ocean: 2700, xt_ocean: 3600)
Coordinates:
    algo      <U8 ...
  * term      (term) <U17 'Q_H' 'Q_H_bar' ... 'Q_star_res' 'Q_star_star_res'
  * time      (time) object 0181-01-01 12:00:00 ... 0181-01-03 12:00:00
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
Data variables:
    evap      (term, time, yt_ocean, xt_ocean) float32 ...
    qh        (term, time, yt_ocean, xt_ocean) float32 ...
    ql        (term, time, yt_ocean, xt_ocean) float32 ...
    taux      (term, time, yt_ocean, xt_ocean) float32 ...
    tauy      (term, time, yt_ocean, xt_ocean) float32 ...